In [2]:
import os 

In [3]:
%pwd 

'c:\\Users\\saurabh\\Desktop\\dl_ops_champios\\vinipro\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\saurabh\\Desktop\\dl_ops_champios\\vinipro'

In [ ]:
# dataclass ---> It imports a python tool called dataclass that helps you create classes 
# that only store data . It reduces the amount of code you need to write (no need to 
# manually write __init__ )  

# Pathlib ---> Imports the Path class , which helps you work with file and folder paths 
# easily and in a clean way for all operationg system .


# @dataclass(frozen=True)  ----> This is a decorator that tells python to dataclass means 
# we cannot change the values once the object is created . It makes the configuration 
# safe from accidental changes . 

In [43]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str 
    local_data_file: Path 
    unzip_dir: Path 

In [44]:
from src.cnnClassifier.constants import * 
from src.cnnClassifier.utils.common import read_yaml , create_directories 

In [45]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 

        create_directories([self.config.artifacts_root]) 

    def get_data_ingestion_config(self):
        config = self.config.data_ingestion 

        create_directories([config.root_dir]) 
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir = config.unzip_dir 

        )

        return data_ingestion_config

In [46]:
import os 
import urllib.request as request 
import zipfile 
from src.cnnClassifier import logger 
from src.cnnClassifier.utils.common import get_size 

In [ ]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} download! with following info: \n {headers}") 
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok=True) 
        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  


In [48]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config() 
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file() 
except Exception as e :
    raise e 

[2025-07-25 18:51:36,449: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-25 18:51:36,459: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-25 18:51:36,470: INFO: common: created directory at: artifacts]
[2025-07-25 18:51:36,476: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-25 18:51:39,023: INFO: 495203984: artifacts/data_ingestion/data.zip download! with following info: 
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B9F0:08CF:9D6C2:1301CC:688384AA
Accept-Ranges: bytes
Date: Fri, 25 Jul 2025 13:21:36 GMT
Via: 1.1 varnish
X-Served-By: cache-del21733-DEL
X-Cache: HIT